In [10]:
import requests
import hashlib
import time
import datetime
from google.colab import userdata
import pandas as pd
import sqlite3

In [11]:
public_key=userdata.get('public_key_marvel')
private_key=userdata.get('private_key_marvel')
timestamp = str(time.time())
md5_hash= timestamp + private_key + public_key
md5_hash = hashlib.md5(md5_hash.encode("utf-8")).hexdigest()

In [ ]:
# Modifique a URL para o endpoint de personagens
url = f"https://gateway.marvel.com/v1/public/events?ts={timestamp}&apikey={public_key}&hash={md5_hash}"
print(url)
# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de personagens: {data['data']['total']}")
    for char in data['data']['results']:
        print(char)

# Criação do banco de dados para os characters

In [68]:
DB_NAME = 'marvel_characters_teste.db'
MAX_RETRIES = 3  # Tentativas por requisição
DELAY = 1  # Atraso entre requisições em segundos

In [11]:
 # Função para pegar todos os personagens
def get_all_characters():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                     'offset': offset
                 }

                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/characters',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()

                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO characters
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['name'],
                         char.get('description', ''),
                         char['modified'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['events']['available'],
                         char['events']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI'],
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os personagens foram armazenados!")
                     return

                 offset += 100
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [79]:
def get_all_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    offset = 0
    total = None
    inserted = 0
    DELAY = 1  # Atraso entre requisições

    while True:
        for attempt in range(MAX_RETRIES):
            try:
                # Autenticação
                ts = str(time.time())
                hash_input = ts + private_key + public_key
                md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                params = {
                    'ts': ts,
                    'apikey': public_key,
                    'hash': md5_hash,
                    'limit': 100,
                    'offset': offset,
                    'orderBy': 'modified'  # Melhora a paginação
                }

                response = requests.get(
                    'https://gateway.marvel.com/v1/public/comics',
                    params=params,
                    timeout=10
                )
                response.raise_for_status()

                data = response.json()
                total = data['data']['total'] if total is None else total
                results = data['data']['results']

                # Inserir quadrinhos no banco
                for comic in results:
                    # Extrair dados das coleções
                    characters_uri = comic.get('characters', {}).get('collectionURI', '')
                    events_uri = comic.get('events', {}).get('collectionURI', '')
                    creators_uri = comic.get('creators', {}).get('collectionURI', '')
                    stories_uri = comic.get('stories', {}).get('collectionURI', '')

                    cursor.execute('''
                        INSERT OR REPLACE INTO comics
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        comic['id'],
                        comic.get('title', 'Sem título'),
                        comic.get('pageCount', 0),
                        comic.get('modified', ''),
                        characters_uri,
                        events_uri,
                        creators_uri,
                        stories_uri
                    ))

                conn.commit()
                inserted += len(results)
                print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                if offset + len(results) >= total:
                    print("\nTodos os quadrinhos foram armazenados!")
                    return

                offset += 100
                time.sleep(DELAY)
                break

            except requests.exceptions.RequestException as e:
                print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                time.sleep(2 ** attempt)
                continue
        else:
            print("Falha após múltiplas tentativas. Abortando...")
            break

    conn.close()

In [106]:
 # Função para pegar todos os personagens
def get_all_events():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 10,
                     'offset': offset
                 }
                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/events',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()
                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO events
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['title'],
                         char.get('description', ''),
                         char['modified'],
                         char['start'],
                         char['end'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['characters']['available'],
                         char['characters']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['creators']['available'],
                         char['creators']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI']
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os eventos foram armazenados!")
                     return

                 offset += 10
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [ ]:
# conn = sqlite3.connect(DB_NAME)
# cursor = conn.cursor()

#     # Tabela principal de personagens
# cursor.execute('''
#     DROP TABLE IF EXISTS events ''')

In [142]:
 # Função para criar o banco
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Tabela principal de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        events_available INTEGER,
        events_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    # Tabela de relacionamento (corrigida)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters_comics (
        id_character INTEGER,
        id_comic INTEGER,
        PRIMARY KEY(id_character, id_comic),
        FOREIGN KEY(id_character) REFERENCES characters(id),
        FOREIGN KEY(id_comic) REFERENCES comics(id)
    )''')

    # Tabela Comics

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS comics (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        page_count INTEGER,
        modified TEXT,
        character_collectionURI TEXT,
        events_collectionURI TEXT,
        creator_collectionURI TEXT,
        stories_collectionURI TEXT
    )''')

    # Tabela Events
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        start TEXT,
        end TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        characters_available INTEGER,
        characters_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        creators_available INTEGER,
        creators_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    # Tabela de personagens com pelo menos 1 evento
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters_with_events (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        events_available INTEGER,
        events_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    conn.commit()
    conn.close()

In [143]:
create_database()
# get_all_comics()
# get_all_characters()
# get_all_events()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} eventos")
conn.close()


Total no banco: 73 eventos


In [152]:
def filtragem_eventos ():

  conn = sqlite3.connect('marvel_characters_teste.db')
  cursor = conn.cursor()

  # Executar consulta com parâmetro
  cursor.execute('''
      INSERT INTO characters_with_events
      SELECT *
      FROM characters
      WHERE events_available > 0
  ''')

  conn.commit()
  conn.close()

In [ ]:
# filtragem_eventos()

conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_with_events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} eventos")
conn.close()

In [136]:
 # Função para pegar todos os personagens
def get_all_events():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 10,
                     'offset': offset
                 }
                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/events',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()
                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO events
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['title'],
                         char.get('description', ''),
                         char['modified'],
                         char['start'],
                         char['end'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                         char['characters']['available'],
                         char['characters']['collectionURI'],
                         char['stories']['available'],
                         char['stories']['collectionURI'],
                         char['creators']['available'],
                         char['creators']['collectionURI'],
                         char['series']['available'],
                         char['series']['collectionURI']
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os eventos foram armazenados!")
                     return

                 offset += 10
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [13]:
# conn = sqlite3.connect(DB_NAME)
# cursor = conn.cursor()

#     # Tabela principal de personagens
# cursor.execute('''
#     DROP TABLE IF EXISTS characters_with_events ''')

In [14]:
 # Função para criar o banco
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Tabela principal de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        events_available INTEGER,
        events_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    # Tabela de relacionamento (corrigida)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS characters_events (
        id_character INTEGER,
        id_event INTEGER,
        PRIMARY KEY(id_character, id_event),
        FOREIGN KEY(id_character) REFERENCES characters(id),
        FOREIGN KEY(id_event) REFERENCES events(id)
    )''')

    # Tabela Comics

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS comics (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        page_count INTEGER,
        modified TEXT,
        character_collectionURI TEXT,
        events_collectionURI TEXT,
        creator_collectionURI TEXT,
        stories_collectionURI TEXT
    )''')

    # Tabela Events
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        description TEXT,
        modified TEXT,
        start TEXT,
        end TEXT,
        comics_available INTEGER,
        comics_collectionURI TEXT,
        characters_available INTEGER,
        characters_collectionURI TEXT,
        stories_available INTEGER,
        stories_collectionURI TEXT,
        creators_available INTEGER,
        creators_collectionURI TEXT,
        series_available INTEGER,
        series_collectionURI TEXT
    )''')

    conn.commit()
    conn.close()

In [24]:
create_database()
# get_all_comics()
# get_all_characters()
# get_all_events()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} eventos")
conn.close()


Total no banco: 73 eventos


In [ ]:
def filtragem_eventos ():

  conn = sqlite3.connect('marvel_characters_teste.db')
  cursor = conn.cursor()

  # Executar consulta com parâmetro
  cursor.execute('''
      INSERT INTO characters_with_events
      SELECT *
      FROM characters
      WHERE events_available > 0
  ''')

  conn.commit()
  conn.close()

In [ ]:
# filtragem_eventos()

conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_with_events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} eventos")
conn.close()

In [ ]:
df_filtragem_eventos = pd.DataFrame(filtragem_eventos())

In [ ]:
df_filtragem_eventos.shape

(696, 12)

# Mostrando o banco marvel_full

In [32]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql('''SELECT name
                        FROM characters
                        WHERE id == 1009718
    ''', conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 1 linhas x 1 colunas


,name
0,Wolverine


In [31]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql('''SELECT c.name, e.title
                        FROM characters c
                        JOIN characters_events ce ON c.id = ce.id_character
                        JOIN events e ON e.id = ce.id_event ''', conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 2556 linhas x 2 colunas


,name,title
0,Abomination (Emil Blonsky),Chaos War
1,Absorbing Man,Acts of Vengeance!
2,Absorbing Man,Civil War
3,Absorbing Man,Fear Itself
4,Absorbing Man,Secret Wars
...,...,...
2551,Ironheart (Riri Williams),Secret Empire
2552,War Machine (James Rhodes),Dark Reign
2553,War Machine (James Rhodes),Initiative
2554,War Machine (James Rhodes),Secret Invasion


In [123]:

conn = sqlite3.connect('marvel_characters_teste.db')
cursor = conn.cursor()

# Executar consulta com parâmetro
cursor.execute('''
    SELECT name, comics_available,series_available,	stories_available,	events_available
    FROM characters
    ORDER BY events_available
    LIMIT 10
''')

# Obter e mostrar resultados
resultados = cursor.fetchall()

In [ ]:
for name, comics_available,series_available,	stories_available,	events_available	 in resultados:
  print(f"- {name}: comics_available: {comics_available} - series_available: {series_available} - stories_available: {stories_available} - events_available: {events_available}")

#Tabelas de Relacionamentos

In [ ]:
# conn = sqlite3.connect(DB_NAME)
# cursor = conn.cursor()

#      # Tabela principal de personagens
# cursor.execute('''
#      DROP TABLE IF EXISTS character_events ''')

In [23]:
import requests
from requests.exceptions import ReadTimeout, RequestException

BASE_TIMEOUT = 10


def character_events():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Contador de requisições
    request_count = 0
    start_time = time.time()

    print("[INÍCIO] Conectado ao banco de dados")

    cursor.execute('SELECT id, events_collectionURI FROM characters WHERE events_available > 0')
    resultados = cursor.fetchall()
    print(f"[BANCO] Total de personagens encontrados: {len(resultados)}")

    for idx, (id_character, events_collection) in enumerate(resultados, 1):
        print(f"\n[PERSONAGEM {idx}/{len(resultados)}] Processando ID: {id_character}")
        print(f"[URL] Endpoint: {events_collection}")

        ts = str(time.time())
        hash_input = ts + private_key + public_key
        md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

        params = {
            'ts': ts,
            'apikey': public_key,
            'hash': md5_hash,
            'limit': 30, #mudar limit?
            'offset': 0
        }

        all_results = []
        total_events = 0

        while True:
            results = None
            for attempt in range(MAX_RETRIES):
                current_timeout = BASE_TIMEOUT * (attempt + 1)
                print(f"\n[TENTATIVA {attempt + 1}/{MAX_RETRIES}]")
                print(f"[TIMEOUT] Configurado para: {current_timeout}s")
                print(f"[PAGINAÇÃO] Offset atual: {params['offset']}")

                try:
                    print("[REQUISIÇÃO] Enviando para API...")
                    # Incrementa o contador a cada tentativa
                    request_count += 1
                    response = requests.get(
                        events_collection,
                        params=params,
                        timeout=current_timeout
                    )
                    print(f"[RESPOSTA] Status Code: {response.status_code}")

                    # Monitoramento do rate limit
                    remaining = int(response.headers.get('X-RateLimit-Remaining', 3000))
                    print(f"[RATE LIMIT] Requisições restantes: {remaining}")

                    response.raise_for_status()
                    data = response.json()

                    if params['offset'] == 0:
                        if data:
                          total_events = data['data']['total']
                          print(f"[EVENTS] Total a processar: {total_events}")

                          page_results = data['data']['results']
                          all_results.extend(page_results)
                          print(f"[PAGINAÇÃO] Eventos nesta página: {len(page_results)}")

                        else:
                            total_events == 0
                    break

                except ReadTimeout as e:
                    print(f"[ERRO] Timeout na tentativa {attempt + 1}: {str(e)}")
                    if attempt == MAX_RETRIES - 1:
                        print("[FALHA] Máximo de tentativas alcançado")
                        break
                    else:
                        wait_time = 2 ** attempt
                        print(f"[AGUARDE] Esperando {wait_time}s para retentativa...")
                        time.sleep(wait_time)

                except RequestException as e:
                    print(f"[ERRO] Falha na requisição: {str(e)}")
                    continue

            # Exibe estatísticas a cada 10 requisições
            if request_count % 10 == 0:
                elapsed = time.time() - start_time
                print(f"\n[ESTATÍSTICAS] Requisições: {request_count} | "
                      f"Tempo decorrido: {elapsed:.2f}s | "
                      f"Req/s: {request_count/elapsed:.2f}")

            if params['offset'] + 30 >= total_events:
                break

            params['offset'] += 30
            time.sleep(0.5)

        if all_results:
            print(f"[BANCO] Inserindo {len(all_results)} eventos para personagem {id_character}")
            try:
                dados = [(id_character, event['id']) for event in all_results]
                cursor.executemany('''
                    INSERT OR IGNORE INTO characters_events
                    VALUES (?, ?)
                ''', dados)
                conn.commit()
            except sqlite3.Error as e:
                print(f"[ERRO BANCO] Falha ao inserir dados: {str(e)}")
                conn.rollback()

    # Relatório final
    total_time = time.time() - start_time
    print(f"\n[RELATÓRIO FINAL]")
    print(f"Total de requisições: {request_count}")
    print(f"Tempo total: {total_time:.2f} segundos")
    print(f"Média de requisições/s: {request_count/total_time:.2f}")

    conn.close()
    print("[BANCO] Conexão fechada")

In [ ]:
character_events()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_events")
print(f"\nTotal no banco: {cursor.fetchone()[0]} linhas")
conn.close()